# Predicting High-Priced Rental Apartments in the USA  
### Milestone 1 – Group_26

---

In [1]:
# Install needed packages if not available
#!pip install ucimlrepo seaborn scikit-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandera as pa
from pandera import Column, DataFrameSchema, Check

from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, ConfusionMatrixDisplay
)

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 5)


## **Summary**

This project investigates whether it is possible to predict when an apartment listing in the United States is **high-priced relative to other listings in the same state**. Using the *Apartment for Rent Classified* dataset from the UCI Machine Learning Repository, we defined a binary target indicating whether each listing's price was above the state-level median. We developed a logistic regression model to classify listings as either high-priced or not. The final classifier achieved an accuracy of 0.7 on unseen data, indicating that it correctly predicts a listing as high-priced in most cases. 

The notebook includes:
- Data loading from the web  
- Data cleaning and wrangling  
- Exploratory data analysis  
- Visualizations  
- Classification modeling  
- Results, discussion, and conclusions  

## Introduction

Homeownership in the United States has become somewhat less attainable for many Americans due to increasing housing prices, rising mortage rates and other increases in the cost of living. As a result, according to recent surveys, around 84% of Gen Z adults report delaying major life milestones in order to afford a home, with many turning to long-term renting instead. Nearly 75% of Gen Z say they prefer renting to owning, which is important to note as this generation represents more than 20% of the U.S. population. Growing rental demand is driving new construction as developers across the country are expected to add more than 500,000 new apartment units across the country.  

These trends lead us to wonder if apartment rental prices vary dramatically across the U.S. A price considered as “expensive” in Texas may be considered “cheap” in New York. As many Gen Z adults are relocating to higher-cost states and more than 75% of them rent, understanding these price differences is increasingly important. Additionally, rental prices are influenced by a variety of listing features. For example, apartments with more bedrooms or bathrooms are generally associated with higher-prices. Other features such as whether pets are allowed or whether the apartment includes additional fees, may also signal higher quality or prices. These features will be incorporated into our model to assess their impact on whether an apartment is classified as high-priced or not. To create a meaningful comparison across regions, we evaluated whether each apartment is **high-priced relative to the median rent within its own state**.

## Research Question

Can a machine learning algorithm accurately predict whether an apartment listing is high-priced relative to the median rental price in its state, using features such as square footage, number of bedrooms and bathrooms, and various listing attributes?

## Dataset

We use the **Apartment for Rent Classified** dataset from the UCI Machine Learning Repository. The dataset contains:
- 10,000+ apartment listings  
- Structural and listing details  
- Geographic information  
- Price and square footage  

---

In [2]:
import pandera as pa

# Data Cleaning & Wrangling

We prepared the dataset by:

1. Loading the data from the UCI ML Repository 
1. Selecting relevant columns  
2. Removing rows with missing or invalid values  
3. Computing the median rental price for each state  
4. Creating a binary target variable `high_price`

In [2]:
# Load dataset directly from the UCI ML Repository
apartment_ds = fetch_ucirepo(id=555)

df = apartment_ds.data.original.copy()

df.head()


/Users/ssemakulapeterwasswa/miniforge3/envs/Group_26/lib/python3.12/site-packages/ucimlrepo/fetch.py:97: DtypeWarning: Columns (0,5,6,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,...,price_display,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
0,5668640009,housing/rent/apartment,One BR 507 & 509 Esplanade,"This unit is located at 507 & 509 Esplanade, R...",NaN,1,1,USD,No,Thumbnail,...,2195,Monthly,542,507 509 Esplanade,Redondo Beach,CA,33.8520,-118.3759,RentLingo,1.577360e+09
1,5668639818,housing/rent/apartment,Three BR 146 Lochview Drive,"This unit is located at 146 Lochview Drive, Ne...",NaN,1.5,3,USD,No,Thumbnail,...,1250,Monthly,1500,146 Lochview Dr,Newport News,VA,37.0867,-76.4941,RentLingo,1.577360e+09
2,5668639686,housing/rent/apartment,Three BR 3101 Morningside Drive,This unit is located at 3101 Morningside Drive...,NaN,2,3,USD,No,Thumbnail,...,1395,Monthly,1650,3101 Morningside Dr,Raleigh,NC,35.8230,-78.6438,RentLingo,1.577360e+09
3,5668639659,housing/rent/apartment,Two BR 209 Aegean Way,"This unit is located at 209 Aegean Way, Vacavi...",NaN,1,2,USD,No,Thumbnail,...,1600,Monthly,820,209 Aegean Way,Vacaville,CA,38.3622,-121.9712,RentLingo,1.577360e+09
4,5668639374,housing/rent/apartment,One BR 4805 Marquette NE,"This unit is located at 4805 Marquette NE, Alb...",NaN,1,1,USD,No,Thumbnail,...,975,Monthly,624,4805 Marquette NE,Albuquerque,NM,35.1038,-106.6110,RentLingo,1.577360e+09


In [3]:
# Data validation (Diana)

In [4]:
# Data validation (Shanze)

In [5]:
# Data validation (Peter)


cols_needed = [
    "price", "square_feet", "bathrooms", "bedrooms",
    "state", "pets_allowed", "fee", "has_photo"
]

df_subset = df[cols_needed].copy()

# Converting numeric columns
df_subset["price"] = pd.to_numeric(df_subset["price"], errors="coerce")
df_subset["square_feet"] = pd.to_numeric(df_subset["square_feet"], errors="coerce")
df_subset["bathrooms"] = pd.to_numeric(df_subset["bathrooms"], errors="coerce")
df_subset["bedrooms"] = pd.to_numeric(df_subset["bedrooms"], errors="coerce")

df_subset.head()


,price,square_feet,bathrooms,bedrooms,state,pets_allowed,fee,has_photo
0,2195.0,542.0,1.0,1.0,CA,Cats,No,Thumbnail
1,1250.0,1500.0,1.5,3.0,VA,"Cats,Dogs",No,Thumbnail
2,1395.0,1650.0,2.0,3.0,NC,NaN,No,Thumbnail
3,1600.0,820.0,1.0,2.0,CA,"Cats,Dogs",No,Thumbnail
4,975.0,624.0,1.0,1.0,NM,"Cats,Dogs",No,Thumbnail


In [6]:
schema = DataFrameSchema(
    {
        "price": Column(
            pa.Float,
            nullable=False,
            checks=[
                Check.greater_than(0),
                Check(lambda s: s.isna().mean() <= 0.05),
            ],
        ),
        "square_feet": Column(
            pa.Float,
            nullable=False,
            checks=[
                Check.greater_than(0),
                Check(lambda s: s.isna().mean() <= 0.05),
            ],
        ),
        "bathrooms": Column(
            pa.Float,
            nullable=False,
            checks=[Check(lambda s: s.isna().mean() <= 0.05)],
        ),
        "bedrooms": Column(
            pa.Float,
            nullable=False,
            checks=[Check(lambda s: s.isna().mean() <= 0.05)],
        ),
        "state": Column(
            pa.String,
            nullable=False,
            checks=[
                Check(lambda s: s.isna().mean() <= 0.05),
                Check.str_length(min_value=1),
            ],
        ),
        "pets_allowed": Column(pa.String, nullable=True),
        "fee": Column(pa.String, nullable=True),
        "has_photo": Column(pa.String, nullable=True),
    },
    coerce=True
)


/Users/ssemakulapeterwasswa/miniforge3/envs/Group_26/lib/python3.12/site-packages/pandera/_pandas_deprecated.py:146: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


In [7]:
# Creating an empty list to keep only valid rows
valid_indices = []

# Looping through each row and validate individually
for idx, row in df_subset.iterrows():
    try:
        schema.validate(row.to_frame().T, lazy=False)  # validating 1 row
        valid_indices.append(idx)  # keeping row if valid
    except pa.errors.SchemaError:
        pass  # drop invalid row

# Keeping only validated clean data
validated_df = df_subset.loc[valid_indices].reset_index(drop=True)

# Overwriting df_subset with clean data
df_subset = validated_df

df_subset.head()


,price,square_feet,bathrooms,bedrooms,state,pets_allowed,fee,has_photo
0,2195.0,542.0,1.0,1.0,CA,Cats,No,Thumbnail
1,1250.0,1500.0,1.5,3.0,VA,"Cats,Dogs",No,Thumbnail
2,1395.0,1650.0,2.0,3.0,NC,NaN,No,Thumbnail
3,1600.0,820.0,1.0,2.0,CA,"Cats,Dogs",No,Thumbnail
4,975.0,624.0,1.0,1.0,NM,"Cats,Dogs",No,Thumbnail


In [ ]:
# Data validation (Grigory)

In [ ]:
cols_needed = [
    "price", "square_feet", "bathrooms", "bedrooms",
    "state", "pets_allowed", "fee", "has_photo"
]

df_subset = df[cols_needed].copy()

# Convert price and square_feet to numeric (coerce invalid values to NaN)
df_subset["price"] = pd.to_numeric(df_subset["price"], errors="coerce")
df_subset["square_feet"] = pd.to_numeric(df_subset["square_feet"], errors="coerce")

# Also convert bathrooms and bedrooms to numeric
df_subset["bathrooms"] = pd.to_numeric(df_subset["bathrooms"], errors="coerce")
df_subset["bedrooms"] = pd.to_numeric(df_subset["bedrooms"], errors="coerce")

# Remove non-positive or missing price/square_feet
df_subset = df_subset[
    (df_subset["price"] > 0) &
    (df_subset["square_feet"] > 0)
]

# Drop rows with missing values in other key areas
df_subset = df_subset.dropna(subset=cols_needed)

# Compute median price per state
state_medians = df_subset.groupby("state")["price"].median()
df_subset["state_median_price"] = df_subset["state"].map(state_medians)

# Create binary target
df_subset["high_price"] = (df_subset["price"] > df_subset["state_median_price"]).astype(int)

df_subset.head()



# Exploratory Data Analysis (EDA)

We explored:

- Summary statistics  
- Class balance  
- Price distribution  
- Relationship between size and price  


In [ ]:
df_subset[["price", "square_feet", "bathrooms"]].describe()

In [ ]:
df_subset["high_price"].value_counts(normalize=True)

In [ ]:
sns.histplot(df_subset["price"], bins=50)
plt.xlim(0, df_subset["price"].quantile(0.99))
plt.title("Figure 1: Distribution of Rental Prices")
plt.show()

In [ ]:
sample_df = df_subset.sample(n=min(5000, len(df_subset)), random_state=123)

sns.scatterplot(
    data=sample_df,
    x="square_feet",
    y="price",
    hue="high_price",
    alpha=0.4
)
plt.xlim(0, sample_df["square_feet"].quantile(0.99))
plt.ylim(0, sample_df["price"].quantile(0.99))
plt.title("Figure 2: Size vs Price (Colored by High-Price Label)")
plt.show()


# Modeling: Logistic Regression Classifier

We predict `high_price` using:

**Numeric features**
- square_feet  
- bathrooms  

**Categorical features**
- bedrooms  
- state  
- pets_allowed  
- fee  
- has_photo  

We use a train-test split and a preprocessing pipeline.


In [ ]:
numeric_features = ["square_feet", "bathrooms"]
categorical_features = ["bedrooms", "state", "pets_allowed", "fee", "has_photo"]

model_df = df_subset[numeric_features + categorical_features + ["high_price"]].copy()

X = model_df[numeric_features + categorical_features]
y = model_df["high_price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=123
)

(len(X_train), len(X_test))


In [ ]:
numeric_transformer = "passthrough"
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

model = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("classifier", LogisticRegression(max_iter=1000))
    ]
)

model.fit(X_train, y_train)


In [ ]:
y_pred = model.predict(X_test)

metrics = {
    "Accuracy": accuracy_score(y_test, y_pred),
    "Precision": precision_score(y_test, y_pred),
    "Recall": recall_score(y_test, y_pred),
    "F1-score": f1_score(y_test, y_pred)
}

pd.DataFrame(metrics, index=["Logistic Regression"])


In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
plt.title("Figure 3: Confusion Matrix")
plt.show()


# Discussion

## Summary of Findings

The logistic regression classifier performed reasonably well in predicting whether an apartment is high-priced relative to its state’s median rental price. Our logistic regression model achieved an accuracy of approximately 0.7, indicating that the model correctly classified 70% of apartment listings as either high-priced or not relative to their state's median rent. The model has a precision score of 0.715 which means the model was correct about 71% of the time when predicting an apartment to be high-priced. The recall score of approximately 0.660 shows the model was able to correctly identify about 66% of actually high-priced listings. Combined, these values resulted in a F1-score of 0.686. 

- Overall, these metrics suggest that the model provides a reasonably balanced performance, though there is room for improvement in correctly identifying all high-priced apartments.

## Analysis

The model performed well when identifying low-priced listings as indicated by the large number of true negatives (2,893). However, it was slightly less successful at identifying high-priced listings misclassifying 1,323 of them as not high-priced when they in fact were. These results align reasonably well with our expectations. We anticipated that apartment prices would vary across the country and the model's high precision of 0.72 suggests that it is usually correct when predicting a listing as high-priced, supporting the idea that certain features contribute meaningfully to the classification. The F1-score of ~0.69 indicates that the model is not overfitting to one class and maintains a balanced trade-off between identifying high-priced listings and avoiding false positives. 

Overall, the results suggest that rental price classification at the state level is predictable to a moderate degree, but not with perfect accuracy. This is likely due to the variability of housing markets across the country. The model performs as expected for a logistic regression approach and provides a useful baseline for predicting whether a listing is high-priced relative to its state’s median rent. 

## Limitations and Future Work
A key limitation of this analysis is that logistic regression may be too simple to capture the full complexity of rental markets, which vary widely across and within states. The model's moderate accuracy (0.70) and recall (0.66) indicate that many high priced listings remain misclassified, suggesting that important predictive features were not included.

Future work could expand this analysis by incorporating more advanced models such as random forests, as well as extracting additional insights through NLP features derived from apartment descriptions. It may also be valuable to explore state by state differences in greater depth and to perform regression modelling to predict exact prices rather than broader categories. Further improvements could come from adding location specific features like the neighbourhood characteristics, building age, or available amenities which may contribute more meaningfully to overall model performance.


# References

1. Investopedia. (2023). Gen Z is having more trouble affording a home — How some are achieving homeownership. https://www.investopedia.com/gen-z-is-having-more-trouble-affording-a-home-how-some-are-achieving-homeownership-11826137

2. Newsweek. (2023). Gen Z is renting, not buying: What it means for the country’s future. https://www.newsweek.com/gen-z-renting-not-buying-what-means-country-future-2120726

3. PR Newswire. (2023). Top 10 states to which Gen Zers are moving and the states they are leaving. https://www.prnewswire.com/news-releases/top-10-states-to-which-gen-zers-are-moving-and-the-states-they-are-leaving-302058380.html

4. Starmer, J. (n.d.). Classification metrics educational videos [YouTube channel]. StatQuest. https://www.youtube.com/user/joshstarmer

5. Scikit-Learn. (n.d.). Logistic regression & preprocessing. https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

6. UCI Machine Learning Repository. (n.d.). Apartment for rent classified dataset. https://archive.ics.uci.edu/ml/datasets/Apartment+for+Rent+Classified